# Koodipaja, 14.12.

Pajassa jatketaan Pandasiin ja DataFrameihin tutustumista. Tässä pajassa luodaan aluksi esimerkkinä siistitympi datasetti, jota sitten käytetään Pandasin toimintojen harjoittelussa.

## Data

Helsingin yliopiston fysiikan ainejärjestöjen opiskelijahuoneella mitattiin viiden Kimble pelin heitot taulukkoon, joka on nimetty `kimble_heitot.txt`. Kukin peli on nimetty tiedoston ylimmän rivin mukaan (92, 93, 94, 95, 100), minkä jälkeen alemmat rivit kuvaavat heiton silmälukua. Tarkoituksena on siistiä/analysoida data niin, että peleistä kerätyt heitot yhdistetään yhdeksi 6x6 taulukoksi, jonka sarake kuvaa silmälukua ja rivi kuvaa kuinka usein silmäluvun jälkeen tuli riviä vastaava silmäluku. Esimerkki taulukko näyttäisi siis esimerkiksi tältä:

| | 1 | 2 | 3 | 4 | 5 | 6 |
|:-|:-|:-|:-|:-|:-|:-|
| "1" | 4 | 7 | 1 | 6 | 6 | 13 |
| "2" | 3 | 3 | 3 | 9 | 6 | 4 |
| "3" | 7 | 5 | 6 | 3 | 6 | 5 |
| "4" | 5 | 4 | 3 | 5 | 4 | 2 |
| "5" | 5 | 8 | 7 | 1 | 1 | 5 |
| "6" | 2 | 2 | 4 | 1 | 0 | 8 |

Taulukkoa voisi nyt lukea niin, että "kun peleissä heitettiin silmäluku 1, niin seuraavaksi heitettiin silmäluku 1 4 kertaa".

### Datan lukeminen

Toisin kuin ensimmäisen pajan esimerkissä, data ei ole nyt suoraan `.csv` muodossa, vaan solujen välissä on tyhjää (whitespace). Taulukko ei myöskään ole niin siisti kuin toivoisi, koska peleissä on heitetty eri määrät heittoja, esim. pelissä "100" heitettiin vain 302 heittoa ja pelissä "92" 455 heittoa. Katsotaan aluksi, miltä tämä näyttää suoraan luettuna:

In [3]:
import pandas as pd
df = pd.read_csv("kimble_heitot.txt", delim_whitespace=True)

# Tarkista df alku ja loppupäät:
df.head(), df.tail()

(   92   93   94   95  100
 0   5  6.0  5.0  5.0  2.0
 1   2  1.0  2.0  2.0  6.0
 2   3  2.0  5.0  5.0  3.0
 3   5  5.0  2.0  4.0  1.0
 4   5  2.0  6.0  3.0  1.0,      92  93  94  95  100
 450   4 NaN NaN NaN  NaN
 451   3 NaN NaN NaN  NaN
 452   5 NaN NaN NaN  NaN
 453   1 NaN NaN NaN  NaN
 454   2 NaN NaN NaN  NaN)

Kuten printistä näkee, taulukko on täytetty pisimmän pelin mittaiseksi ja lyhyemmät pelit on täytetty `NaN` (Not a Number) arvoilla. Tämä on ongelma, koska Pandas ei osaa käsitellä `NaN` arvoja. Tässä tapauksessa `NaN` arvot voi esimerkiksi korvata nollilla, jotta Pandas osaisi käsitellä taulukkoa. Tämä onnistuu Pandasin `fillna` toiminnolla, joka korvaa `NaN` arvot annetulla arvolla.

In [4]:
df = df.fillna(0)
df.tail()

,92,93,94,95,100
450,4,0.0,0.0,0.0,0.0
451,3,0.0,0.0,0.0,0.0
452,5,0.0,0.0,0.0,0.0
453,1,0.0,0.0,0.0,0.0
454,2,0.0,0.0,0.0,0.0


Taulukkoon nyt myös eksynyt mukaan desimaalilukuja, joten siisteyden takia vaihdetaan koko taulukko kokonaisluvuiksi:

In [6]:
df = df.astype(int)
df.tail()

,92,93,94,95,100
450,4,0,0,0,0
451,3,0,0,0,0
452,5,0,0,0,0
453,1,0,0,0,0
454,2,0,0,0,0


## Datan käsittely ja tallennus

Nyt kun data on luettu ja siistitty, voidaan siirtyä Pandasin toimintojen harjoitteluun. Tässä harjoituksessa tarkoituksena on tiivistää dataa niin, että saadaan yksi taulukko, jossa on kuvattu kuinka usein kukin silmäluku tuli seuraavaksi silmäluvun jälkeen. Taulukko tulee siis olemaan aluksi kolmiulotteinen, yksi ulottuvuus kuvaa pelinumeroa, yksi nykyistä silmälukua ja yksi tulevaa silmälukua. Tämän jälkeen taulukko tiivistetään yhdeksi taulukoksi, jossa on kuvattu kuinka usein kukin silmäluku tuli seuraavaksi silmäluvun jälkeen ja pelinumero jätetään huomiotta.

### Numpy-taulukko

Käydään nyt kaikki taulukot läpi, ja lasketaan ns. vanhan liiton for-loopilla kuinka usein kukin silmäluku tuli seuraavaksi silmäluvun jälkeen. Ajatuksena on alustaa aluksi haluttu tyhjä 6x6 taulukko, johon sitten lisätään oikeaan kohtaan yksi, kun silmäluku tulee seuraavaksi silmäluvun jälkeen.

In [20]:
import numpy as np

luvut = np.zeros((6,6), dtype=int) # 6x6 taulukko nollilla (kokonaislukuja)

# Käydään läpi kaikki sarakkeet
for rivi in df:
    # Lähdetään liikkeellee ensimmäisestä rivistä
    aikaisempi = df[rivi][0]

    for i in df[rivi][1:]:
        # Pelien pituus vaihtelee, mutta tiedetään että peli joko päättyy rivin loppuun tai silmälukuun 0
        if i == 0:
            break

        # Kasvatetaan taulukon arvoa yhdellä (indeksi alkavat nollasta)
        luvut[aikaisempi-1, i-1] = luvut[aikaisempi-1, i-1] + 1
        aikaisempi = i

# Lopputulos
luvut

array([[39, 49, 63, 56, 49, 91],
       [54, 25, 36, 50, 82, 33],
       [50, 40, 30, 99, 49, 40],
       [67, 38, 96, 32, 53, 46],
       [45, 92, 51, 39, 38, 61],
       [93, 36, 33, 56, 53, 26]])

### Pandas-taulukko ja tallennus

Nyt, kun meillä on Numpy-taulukko sen voisi oikeastaan kirjoitaa suoraan tiedostoon ja tallentaa tuloksen Numpyn `np.savetxt` toiminnolla. Tässä harjoituksessa kuitenkin halutaan käyttää Pandasia, joten muutetaan Numpy-taulukko Pandasin DataFrameksi ja käytetään Pandasin `to_csv` toimintoa tiedoston kirjoittamiseen.

In [22]:
tulos = pd.DataFrame(luvut, columns=["1", "2", "3", "4", "5", "6"], index=["1", "2", "3", "4", "5", "6"])
tulos.to_csv("kimble_tulos.csv")
tulos

,1,2,3,4,5,6
1,39,49,63,56,49,91
2,54,25,36,50,82,33
3,50,40,30,99,49,40
4,67,38,96,32,53,46
5,45,92,51,39,38,61
6,93,36,33,56,53,26
